In [8]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from numpy.lib.stride_tricks import as_strided as stride
from sklearn.model_selection import GroupKFold,StratifiedKFold,GroupShuffleSplit
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import random,gc
from tqdm import tqdm

# 特征提取

In [9]:
# xyt三维特征
def neighbor3d_feature(group):
    thresholds = [24,48,96]
    dist = np.zeros((group.shape[0],group.shape[0]), dtype='float32')
    tmp = group[['x','y','t']].values.astype('float32').reshape(-1,3)
    dx,dy,dt = (tmp[:,0].reshape(-1,1)-tmp[:,0].reshape(1,-1))**2,(tmp[:,1].reshape(-1,1)-tmp[:,1].reshape(1,-1))**2,(tmp[:,2].reshape(-1,1)-tmp[:,2].reshape(1,-1))**2
    dist = np.sqrt(dx+dy+dt)
    tmps = []
    for t in thresholds:
        isInBall = dist<=t
        num = dist.sum(axis=1)
        q = group['q'].values.reshape(1,-1) * isInBall
        d = dist * isInBall
        n = isInBall.sum(axis=1)
        tmps += [n, d.sum(axis=1)/n, q.sum(axis=1)/n, q.max(axis=1)]
        q[q==0],d[d==0] = np.inf,np.inf
        dmin = d.min(axis=1)
        dmin[dmin==np.inf] = -1
        tmps += [dmin, q.min(axis=1)]
    return np.hstack([i.reshape(-1,1) for i in tmps])

# xy二维特征
def neighbor2d_feature(group):
    thresholds = [10*np.sqrt(2), 20*np.sqrt(2), 40*np.sqrt(2)]
    dist = np.zeros((group.shape[0],group.shape[0]), dtype='float32')
    tmp = group[['x','y']].values.astype('float32').reshape(-1,2)
    dx,dy = (tmp[:,0].reshape(-1,1)-tmp[:,0].reshape(1,-1))**2,(tmp[:,1].reshape(-1,1)-tmp[:,1].reshape(1,-1))**2
    dist = np.sqrt(dx+dy)
    tmps = []
    for t in thresholds:
        isInBall = dist<=t
        num = dist.sum(axis=1)
        q = group['q'].values.reshape(1,-1) * isInBall
        d = dist * isInBall
        n = isInBall.sum(axis=1)
        tmps += [n, d.sum(axis=1)/n, q.sum(axis=1)/n, q.max(axis=1)]
        q[q==0],d[d==0] = np.inf,np.inf
        dmin = d.min(axis=1)
        dmin[dmin==np.inf] = -1
        tmps += [dmin, q.min(axis=1)]
    return np.hstack([i.reshape(-1,1) for i in tmps])

In [10]:
# 特征提取函数
def feature_extract(data):
    out = pd.DataFrame(dtype='float32')
    # base feature
    print('Start to extract the base features...')
    
#  初赛结束后新加的部分特征，线下结果有较大提升    
#     out['angle'] = np.arctan(data['y'] / (data['x'] + 1e-5))
#     out['c_angle'] = np.arctan(data['ycmc'] / (data['xcmc'] + 1e-5))
#     out['q_energymc_ratio'] = data['q'] / (data['energymc'] + 1e-5)
#     data['q_r'] = round(data['q'])
#     for f in tqdm(['x', 'y', 'terror', 'q_r']):
#         out[f + '_count'] = data[f].map(data[f].value_counts())
#     for f in tqdm([
#         ['x', 'y'], ['x', 'terror'], ['x', 'q_r'],
#         ['y', 'terror'], ['y', 'q_r'], ['terror', 'q_r']
#         ]):
#         out['_'.join(f) + '_count'] = data.groupby(f)['hit_id'].transform('count')
#     out['event_id_t_mm'] = data.groupby('event_id')['t'].transform('max') - data.groupby('event_id')['t'].transform('min')
#     out['t_gap'] = (data['t']-data.groupby('event_id')['t'].transform('min')) / out['event_id_t_mm']
#     out['event_id_q_sum'] = data.groupby('event_id')['q'].transform('sum')
#     out['event_id_q_sum_energymc_diff'] = data['energymc'] - out['event_id_q_sum']
#     out['event_id_q_sum_energymc_prop'] = out['event_id_q_sum'] / (data['energymc'] + 1e-5)
    
    out['x_'],out['y_'] = data['x'],data['y']
    out['t_'] = data['t']
    
    data['x'],data['y'] = data['x']-data['xcmc'], data['y']-data['ycmc']
    data['t'] = data['t'] - data.groupby('event_id')['t'].transform('mean')
    
    out['x'],out['y'] = data['x'],data['y']
    out['t'] = data['t']
    
    data['tmin'],out['tmin'] = data['t']-data['terror'],data['t']-data['terror']
    data['tmax'],out['tmax'] = data['t']+data['terror'],data['t']+data['terror']
    out['x/t'] = data['x']/data['t']
    out['y/t'] = data['y']/data['t']
    data['d'] = (data['x']**2+data['y']**2)*np.sin(np.pi/2 - data['thetamc']*np.pi/180)
    out['d'] = data['d']
    out['d/t'] = out['d']/data['t']
    out['d/tmin'] = out['d']/out['tmin']
    out['d/tmax'] = out['d']/out['tmax']
    
    out['q'] = data['q']
    out['q/t'] = data['q']/out['t']
    out['q/tmin'] = data['q']/out['tmin']
    out['q/tmax'] = data['q']/out['tmax']
    
    data_groupby_eid = data.groupby('event_id')
    out['x_diff1'] = data_groupby_eid['x'].diff(1)
    out['y_diff1'] = data_groupby_eid['y'].diff(1)
    out['d_diff1'] = data_groupby_eid['d'].diff(1)
    out['d_diff2'] = data_groupby_eid['d'].diff(2)
    out['d_diff4'] = data_groupby_eid['d'].diff(4)
    out['d_diff8'] = data_groupby_eid['d'].diff(8)
    out['t_diff1'] = data_groupby_eid['t'].diff(1)
    out['tmin_diff1'] = data_groupby_eid['tmin'].diff(1)
    out['tmax_diff1'] = data_groupby_eid['tmax'].diff(1)
    out['t_diff2'] = data_groupby_eid['t'].diff(2)
    out['tmin_diff2'] = data_groupby_eid['tmin'].diff(2)
    out['tmax_diff2'] = data_groupby_eid['tmax'].diff(2)
    out['t_diff4'] = data_groupby_eid['t'].diff(4)
    out['tmin_diff4'] = data_groupby_eid['tmin'].diff(4)
    out['tmax_diff4'] = data_groupby_eid['tmax'].diff(4)
    out['t_diff8'] = data_groupby_eid['t'].diff(8)
    out['tmin_diff8'] = data_groupby_eid['tmin'].diff(8)
    out['tmax_diff8'] = data_groupby_eid['tmax'].diff(8)
    tmps = []
    for name,group in data_groupby_eid:
        tmps.append( (group['q'] / group['q'].sum()).values )
    out['q_ratio'] = np.hstack(tmps).reshape(-1,1)
    out['energy_q_ratio'] = out['q_ratio']*data['energymc']
    
    out['nhit'] = data['nhit']
    out['nhitreal'] = data['nhitreal']
    out['nhitreal_ratio'] = data['nhitreal'] / data['nhit']
    out['energymc'] = data['energymc']
    out['thetamc'] = data['thetamc']

    out['terror'] = data['terror']
    out['phimc'] = data['phimc']
    out['xcmc'],out['ycmc'] = data['xcmc'],data['ycmc']
    
    # neighbour feature
    print('Start to extract the neighbor features...')
    tmps = []
    for name,group in tqdm(data.groupby('event_id')):
        tmps.append( neighbor2d_feature(group) )
    feature_append1 = pd.DataFrame(np.vstack(tmps), columns=['neighbor1_num_2d', 'neighbor1_d_mean_2d', 'neighbor1_q_mean_2d', 'neighbor1_q_max_2d', 'neighbor1_d_min_2d', 'neighbor1_q_min_2d', 
                                                             'neighbor2_num_2d', 'neighbor2_d_mean_2d', 'neighbor2_q_mean_2d', 'neighbor2_q_max_2d', 'neighbor2_d_min_2d', 'neighbor2_q_min_2d',
                                                             'neighbor3_num_2d', 'neighbor3_d_mean_2d', 'neighbor3_q_mean_2d', 'neighbor3_q_max_2d', 'neighbor3_d_min_2d', 'neighbor3_q_min_2d',],
                                   dtype='float32')
    tmps = []
    for name,group in tqdm(data.groupby('event_id')):
        tmps.append( neighbor3d_feature(group) )
    feature_append2 = pd.DataFrame(np.vstack(tmps), columns=['neighbor1_num_3d', 'neighbor1_d_mean_3d', 'neighbor1_q_mean_3d', 'neighbor1_q_max_3d', 'neighbor1_d_min_3d', 'neighbor1_q_min_3d', 
                                                             'neighbor2_num_3d', 'neighbor2_d_mean_3d', 'neighbor2_q_mean_3d', 'neighbor2_q_max_3d', 'neighbor2_d_min_3d', 'neighbor2_q_min_3d',
                                                             'neighbor3_num_3d', 'neighbor3_d_mean_3d', 'neighbor3_q_mean_3d', 'neighbor3_q_max_3d', 'neighbor3_d_min_3d', 'neighbor3_q_min_3d',],
                                   dtype='float32')
    
    out = pd.concat([out,feature_append1,feature_append2], axis=1)
    if 'flag' in data:
        out['flag'] = data['flag']
    return out

## 特征提取与数据保存

In [ ]:
h5 = pd.HDFStore('data.h5','w', complevel=4, complib='blosc')

#训练集特征
print('train data...')
data_train = pd.merge(pd.read_csv('data/train.csv'), pd.read_csv('data/event.csv')).sort_values(by=['event_id', 't']).reset_index(drop=True).astype('float32')
train = feature_extract(data_train)
h5['train'] = train

#测试集特征
print('test data...')
data_test = pd.merge(pd.read_csv('data/test.csv'), pd.read_csv('data/event.csv')).sort_values(by=['event_id', 't']).reset_index(drop=True).astype('float32')
test = feature_extract(data_test)
h5['test'] = test

del data_train,data_test,train,test
gc.collect()
h5.close()

# 模型训练

In [12]:
# 数据读取
train = pd.read_hdf('data.h5', key='train').astype('float32')
test  = pd.read_hdf('data.h5', key='test').astype('float32')

In [13]:
# 标签数据
groupId = pd.read_csv('data/train.csv', usecols=['event_id','t']).sort_values(by=['event_id', 't']).reset_index(drop=True)

In [ ]:
# 初赛结束后新加的编码特征，结果有较大提升

# train['detector'] = train['x_'].astype('str') + '_' + train['y_'].astype('str')
# test['detector'] = test['x_'].astype('str') + '_' + test['y_'].astype('str')

# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1029)
# for f in tqdm(['detector', 'terror']):
#     train[f + '_target_enc'] = 0
#     test[f + '_target_enc'] = 0
#     for i, (trn_idx, val_idx) in enumerate(skf.split(train, train['flag'])):
#         trn_x = train[[f, 'flag']].iloc[trn_idx].reset_index(drop=True)
#         val_x = train[[f]].iloc[val_idx].reset_index(drop=True)
#         enc_df = trn_x.groupby(f, as_index=False)['flag'].agg({f + '_target_enc': 'mean'})
#         val_x = val_x.merge(enc_df, on=f, how='left')
#         test_x = test[[f]].merge(enc_df, on=f, how='left')
#         val_x[f + '_target_enc'] = val_x[f + '_target_enc'].fillna(train['flag'].mean())
#         test_x[f + '_target_enc'] = test_x[f + '_target_enc'].fillna(train['flag'].mean())
#         train.loc[val_idx, f + '_target_enc'] = val_x[f + '_target_enc'].values
#         test[f + '_target_enc'] += test_x[f + '_target_enc'].values / skf.n_splits

# del trn_x, val_x, enc_df,test_x
# gc.collect()

In [14]:
# 模型参数
params = {
            'learning_rate': 0.2,
            'metric': 'auc',
            'objective': 'binary',
            'n_jobs': -1,
            'seed': 1029,
            'max_depth': -1,
            'num_leaves': 64,
            #'lambda_l1': 0.5,
            'lambda_l2': 0.5
        }

In [ ]:
# 模型训练
folds = GroupKFold(n_splits=3)
oof_pred = np.zeros((len(train), ))
y_pred = np.zeros((len(test), ))

# drop_cols = ['flag', 'detector']
drop_cols = ['flag']
feaImportances = []
for fold, (tr_ind, val_ind) in enumerate(folds.split(train, train['flag'], groupId['event_id'])):
    print(f'Fold {fold + 1}')
    x_train, x_val = train.drop(drop_cols, axis=1).iloc[tr_ind].astype('float32'), train.drop(drop_cols, axis=1).iloc[val_ind].astype('float32')
    y_train, y_val = train['flag'].iloc[tr_ind].astype('float32'), train['flag'].iloc[val_ind].astype('float32')
    lgb_train,lgb_test = lgb.Dataset(x_train, y_train),lgb.Dataset(x_val, y_val)
    model = lgb.train(params, 
                      lgb_train, 
                      num_boost_round=2000,
                      early_stopping_rounds=35,
                      valid_sets=[lgb_test],
                      verbose_eval=100)
    oof_pred[val_ind] = model.predict(x_val)
    y_pred += model.predict(test.drop(drop_cols[1:], axis=1)) / folds.n_splits
    feaImportances.append(pd.DataFrame({'feature': model.feature_name(), 
                                        'importance':model.feature_importance()}).sort_values('importance', ascending=False))
    del x_train, x_val, y_train, y_val, lgb_train, lgb_test
    gc.collect()

score = roc_auc_score(train['flag'], oof_pred) 
print('auc: ', score)

# 生成提交文件

In [ ]:
result = pd.merge(pd.read_csv('data/test.csv').sort_values(by=['event_id', 't']), pd.read_csv('data/event.csv'))
result['flag_pred'] = y_pred
submission = result[['hit_id', 'flag_pred', 'event_id']]

for eve_id,group in tqdm(result.groupby(['event_id'])[['nhitreal','flag_pred']]):
    submission['flag_pred'].iloc[group['flag_pred'].nlargest(group['nhitreal'].iloc[0].astype('int')+5).index] = 1

submission['flag_pred'][submission['flag_pred']!=1] = 0

submission = submission.sort_values(by='hit_id')
submission.to_csv(f'submit/submit_lgb_{score}.csv', index=False)
submission.flag_pred.value_counts()